In [1]:
import numpy as np
import pandas as pd
from scipy import special
from random import sample
import itertools
from datetime import timedelta
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import math

from dateutil.relativedelta import *
from datetime import date
from datetime import datetime


def formatarStr (DataFrame, columns):
    for col in columns:
        DataFrame[col] = DataFrame[col].str.strip()
        
def calcularIdade(dtNasc):
    try:
        if dtNasc == 0: 
            return np.nan
        else: 
            today = date.today()
            dob = datetime.strptime(str(dtNasc), '%d/%m/%Y').date()
            age = relativedelta(today, dob)
            return age.years
    except (RuntimeError, TypeError, NameError, ValueError):
        return np.nan

def calcularAnoMes(dtAcid): 
    if len(dtAcid) == 10:
        anoMesStr = dtAcid[6:]+dtAcid[3:5]
        return anoMesStr
    elif len(dtAcid) == 7:
        anoMesStr = dtAcid.replace('/', '')
        return anoMesStr
    else:
        return dtAcid
    

In [2]:
#nomeArquivoOrigem = 'datasets/origem/cat-jul-ago-set-2018.csv'
#nomeArquivoResultado2018 = 'datasets/result/dados_cat_201803.csv'
#nomeArquivoResultado2017 = 'datasets/result/dados_cat_2017A.csv'

nomeArquivoOrigem = 'datasets/origem/cat-comp-outnovdez-2018.csv'
nomeArquivoResultado2018 = 'datasets/result/dados_cat_201804.csv'
nomeArquivoResultado2017 = 'datasets/result/dados_cat_2017B.csv'


columns_types = {'agente': str, 'dt_acidente': str, 'cbo': int, 'cbo_desc': str, 'cid': str, 'cid_desc': str, 
                 'cnae': int, 'cnae_desc': str, 'emitente': str, 'especie': str, 'filiacao': str, 
                 'obito': str, 'munic_emp': str, 'natureza': str, 'origem': str, 'parte': str, 'sexo': str, 
                 'tipo_acidente': str, 'uf_acidente': str, 'uf_emp': str, 'dt_afast': str, 'dt_despacho': str, 'dt_acid': str, 
                 'dt_nasc': str, 'dt_emissao': str}

nomes  = ['agente', 'dt_acidente', 'cbo', 'cbo_desc', 'cid', 'cid_desc', 'cnae', 'cnae_desc', 'emitente', 'especie', 
          'filiacao', 'obito', 'munic_emp', 'natureza', 'origem', 'parte', 'sexo', 'tipo_acidente', 'uf_acidente', 'uf_emp', 
          'dt_afast', 'dt_despacho', 'dt_acid', 'dt_nasc', 'dt_emissao']


dados_originais = pd.read_csv(nomeArquivoOrigem, sep = ';', header=0, names=nomes, dtype=columns_types, encoding='iso-8859-1')

df_original = pd.DataFrame(data = dados_originais, columns=nomes)
df_original.shape[0] #117439

119218

In [3]:
df_original.head()

,agente,dt_acidente,cbo,cbo_desc,cid,cid_desc,cnae,cnae_desc,emitente,especie,...,parte,sexo,tipo_acidente,uf_acidente,uf_emp,dt_afast,dt_despacho,dt_acid,dt_nasc,dt_emissao
0,"Maquina, Nic",2018/01,848520,848520-Magarefe,S927,S92.7 Frat Mult do P,1012,"Abate de Suinos, Ave",Empregador,Pa,...,Artelho,Masculino,Típico,Roraima,Paraná,2018/09,0000/00,11/01/2018,14/08/1968,02/10/2018
1,"Esforco Excessivo, N",2018/01,514320,{ñ class},G560,G56.0 Sindr do Tunel,8610,Atividades de Atendi,Empregador,Pa,...,Punho,Feminino,Doença,{ñ class},Rio Grande do Sul,2018/10,0000/00,12/01/2018,12/02/1978,30/10/2018
2,"Veiculo, Nic",2018/01,414110,414110-Armazenista,S920,S92.0 Frat do Calcan,4930,Transporte Rodoviari,Empregador,Pa,...,Pe (Exceto Artelhos),Feminino,Típico,Maranhão,São Paulo,2018/10,0000/00,26/01/2018,24/02/1993,26/10/2018
3,"Asfalto, Alcatrao, P",2018/01,514205,514205-Coletor Lixo,M255,M25.5 Dor Articular,3811,Coleta de Residuos N,Empregador,Pa,...,"Perna (Do Tornozelo,",Masculino,Típico,Maranhão,São Paulo,2018/10,0000/00,21/01/2018,30/04/1990,31/10/2018
4,Veiculo Rodoviario M,2018/02,782405,782405-Motorista de,T920,T92.0 Sequelas de Fe,4921,Transporte Rodoviari,Empregador,Pa,...,Antebraco (Entre O P,Masculino,Típico,{ñ class},Mato Grosso,2018/10,0000/00,10/02/2018,12/01/1977,18/10/2018


In [4]:
df_cat = pd.DataFrame(df_original[['agente', 'dt_acidente', 'cbo', 'cid', 'cnae', 'obito', 'natureza', 'parte','sexo', 'tipo_acidente', 
                   'uf_emp', 'dt_nasc']])

formatarStr(df_cat, ['agente', 'dt_acidente', 'cid', 'obito', 'natureza', 'parte', 'sexo', 'tipo_acidente', 'uf_emp', 
                     'dt_nasc'])
df_cat.head(10)

,agente,dt_acidente,cbo,cid,cnae,obito,natureza,parte,sexo,tipo_acidente,uf_emp,dt_nasc
0,"Maquina, Nic",2018/01,848520,S927,1012,Não,Fratura,Artelho,Masculino,Típico,Paraná,14/08/1968
1,"Esforco Excessivo, N",2018/01,514320,G560,8610,Não,"Doenca, Nic",Punho,Feminino,Doença,Rio Grande do Sul,12/02/1978
2,"Veiculo, Nic",2018/01,414110,S920,4930,Não,Fratura,Pe (Exceto Artelhos),Feminino,Típico,São Paulo,24/02/1993
3,"Asfalto, Alcatrao, P",2018/01,514205,M255,3811,Não,"Distensao, Torcao","Perna (Do Tornozelo,",Masculino,Típico,São Paulo,30/04/1990
4,Veiculo Rodoviario M,2018/02,782405,T920,4921,Não,Fratura,Antebraco (Entre O P,Masculino,Típico,Mato Grosso,12/01/1977
5,Prensa - Maquina,2018/02,724515,S678,2593,Não,Amputacao ou Enuclea,Dedo,Feminino,Típico,São Paulo,07/08/1985
6,Bicicleta,2018/02,914405,M849,4520,Não,Fratura,Perna (Entre O Torno,Masculino,Trajeto,Minas Gerais,05/04/1999
7,Chao - Superficie Ut,2018/02,782820,M544,8621,Não,"Distensao, Torcao",Dorso (Inclusive Mus,Masculino,Típico,São Paulo,01/12/1981
8,Metal - Inclui Liga,2018/03,821440,W456,3011,Não,"Corte, Laceracao, Fe",Antebraco (Entre O P,Masculino,Típico,Rio de Janeiro,09/10/1961
9,Carvao,2018/03,711130,S822,500,Não,Fratura,"Perna (Do Tornozelo,",Masculino,Típico,Santa Catarina,28/12/1979


In [5]:
print('Dados ausentes, por coluna: ')
# verificar se, nas colunas mais importantes, há registros com o valor {ñ class}:
f_agente = df_cat[df_cat['agente'] == '{ñ class}'].shape[0] #
print('agente: {}'.format(f_agente))
f_cbo = df_cat[df_cat['cbo'] == '{ñ class}'].shape[0]  #
print('cbo: {}'.format(f_cbo))
f_cid = df_cat[df_cat['cid'] == '{ñ class}'].shape[0]  #
print('cid: {}'.format(f_cid))
f_cnae = df_cat[df_cat['cnae'] == '{ñ class}'].shape[0]  #
print('cnae: {}'.format(f_cnae))
f_obito = df_cat[df_cat['obito'] == '{ñ class}'].shape[0]  #
print('obito: {}'.format(f_obito))
f_natureza = df_cat[df_cat['natureza'] == '{ñ class}'].shape[0]  #
print('natureza: {}'.format(f_natureza))
f_parte = df_cat[df_cat['parte'] == '{ñ class}'].shape[0]  # 
print('parte: {}'.format(f_parte))
f_sexo = df_cat[df_cat['sexo'] == '{ñ class}'].shape[0]  #
print('sexo: {}'.format(f_sexo))
f_tipo = df_cat[df_cat['tipo_acidente'] == '{ñ class}'].shape[0]  #
print('tipo: {}'.format(f_tipo))
f_uf = df_cat[df_cat['uf_emp'] == '{ñ class}'].shape[0]  #
print('uf: {}'.format(f_uf))

#se necessário, apagar as linhas cujo valor de algum campo importante seja nulo
df_cat.drop(df_cat[df_cat['parte'] == '{ñ class}'].index, inplace = True)

df_cat.shape[0] #117169

df_cat.info()

Dados ausentes, por coluna: 
agente: 70
cbo: 0
cid: 0
cnae: 0
obito: 0
natureza: 64
parte: 2
sexo: 0
tipo: 0
uf: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 119216 entries, 0 to 119217
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   agente         119216 non-null  object
 1   dt_acidente    119216 non-null  object
 2   cbo            119216 non-null  int32 
 3   cid            119216 non-null  object
 4   cnae           119216 non-null  int32 
 5   obito          119216 non-null  object
 6   natureza       119216 non-null  object
 7   parte          119216 non-null  object
 8   sexo           119216 non-null  object
 9   tipo_acidente  119216 non-null  object
 10  uf_emp         119216 non-null  object
 11  dt_nasc        119216 non-null  object
dtypes: int32(2), object(10)
memory usage: 10.9+ MB


In [6]:
#tratamento dos valores nulos de 'dt_nasc':
df_cat.fillna(0, inplace = True)
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119216 entries, 0 to 119217
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   agente         119216 non-null  object
 1   dt_acidente    119216 non-null  object
 2   cbo            119216 non-null  int32 
 3   cid            119216 non-null  object
 4   cnae           119216 non-null  int32 
 5   obito          119216 non-null  object
 6   natureza       119216 non-null  object
 7   parte          119216 non-null  object
 8   sexo           119216 non-null  object
 9   tipo_acidente  119216 non-null  object
 10  uf_emp         119216 non-null  object
 11  dt_nasc        119216 non-null  object
dtypes: int32(2), object(10)
memory usage: 10.9+ MB


In [7]:
df_cat['idade'] = df_cat['dt_nasc'].apply(lambda x: calcularIdade(x))
df_cat.head(5)

,agente,dt_acidente,cbo,cid,cnae,obito,natureza,parte,sexo,tipo_acidente,uf_emp,dt_nasc,idade
0,"Maquina, Nic",2018/01,848520,S927,1012,Não,Fratura,Artelho,Masculino,Típico,Paraná,14/08/1968,52.0
1,"Esforco Excessivo, N",2018/01,514320,G560,8610,Não,"Doenca, Nic",Punho,Feminino,Doença,Rio Grande do Sul,12/02/1978,43.0
2,"Veiculo, Nic",2018/01,414110,S920,4930,Não,Fratura,Pe (Exceto Artelhos),Feminino,Típico,São Paulo,24/02/1993,28.0
3,"Asfalto, Alcatrao, P",2018/01,514205,M255,3811,Não,"Distensao, Torcao","Perna (Do Tornozelo,",Masculino,Típico,São Paulo,30/04/1990,30.0
4,Veiculo Rodoviario M,2018/02,782405,T920,4921,Não,Fratura,Antebraco (Entre O P,Masculino,Típico,Mato Grosso,12/01/1977,44.0


In [8]:
df_cat[df_cat['idade'].isnull()].shape[0]  
# 81 campos faltantes na coluna, por não haver dt_nasc

2

In [9]:
df_cat[df_cat['idade'].isnull()].head(5)
# 1158
# 2681

,agente,dt_acidente,cbo,cid,cnae,obito,natureza,parte,sexo,tipo_acidente,uf_emp,dt_nasc,idade
50645,"Faca, Facao- Ferrame",2018/11,322205,Z209,8610,Não,"Lesao Imediata, Nic",Dedo,Feminino,Típico,Santa Catarina,{ñ class},NaN
81215,Produto Biologico (S,2018/11,223147,Z209,8660,Não,"Corte, Laceracao, Fe",Dedo,Masculino,Típico,Rio de Janeiro,{ñ class},NaN


In [10]:
mediaIdades = math.floor(df_cat['idade'].mean())
mediaIdades

37

In [11]:
df_cat.fillna(mediaIdades, inplace=True)

df_cat.isnull().sum()

agente           0
dt_acidente      0
cbo              0
cid              0
cnae             0
obito            0
natureza         0
parte            0
sexo             0
tipo_acidente    0
uf_emp           0
dt_nasc          0
idade            0
dtype: int64

In [12]:
df_cat.columns.name = 'caracteristica'
df_cat.index.name = "id"
df_cat[df_cat.index == 1158]

caracteristica,agente,dt_acidente,cbo,cid,cnae,obito,natureza,parte,sexo,tipo_acidente,uf_emp,dt_nasc,idade
id,,,,,,,,,,,,,
1158,Animal Vivo,2018/09,519940,S921,3600,Não,Fratura,Pe (Exceto Artelhos),Feminino,Típico,Paraná,14/04/1993,27.0


In [13]:
#criar coluna anoMes e deletar a dt_acidente
df_cat['anoMes'] = df_cat['dt_acidente'].apply(lambda x: calcularAnoMes(x))
df_cat.head()

caracteristica,agente,dt_acidente,cbo,cid,cnae,obito,natureza,parte,sexo,tipo_acidente,uf_emp,dt_nasc,idade,anoMes
id,,,,,,,,,,,,,,
0,"Maquina, Nic",2018/01,848520,S927,1012,Não,Fratura,Artelho,Masculino,Típico,Paraná,14/08/1968,52.0,201801
1,"Esforco Excessivo, N",2018/01,514320,G560,8610,Não,"Doenca, Nic",Punho,Feminino,Doença,Rio Grande do Sul,12/02/1978,43.0,201801
2,"Veiculo, Nic",2018/01,414110,S920,4930,Não,Fratura,Pe (Exceto Artelhos),Feminino,Típico,São Paulo,24/02/1993,28.0,201801
3,"Asfalto, Alcatrao, P",2018/01,514205,M255,3811,Não,"Distensao, Torcao","Perna (Do Tornozelo,",Masculino,Típico,São Paulo,30/04/1990,30.0,201801
4,Veiculo Rodoviario M,2018/02,782405,T920,4921,Não,Fratura,Antebraco (Entre O P,Masculino,Típico,Mato Grosso,12/01/1977,44.0,201802


In [14]:
df_cat.drop(['dt_acidente', 'dt_nasc'], axis=1, inplace = True)
df_cat.head()

caracteristica,agente,cbo,cid,cnae,obito,natureza,parte,sexo,tipo_acidente,uf_emp,idade,anoMes
id,,,,,,,,,,,,
0,"Maquina, Nic",848520,S927,1012,Não,Fratura,Artelho,Masculino,Típico,Paraná,52.0,201801
1,"Esforco Excessivo, N",514320,G560,8610,Não,"Doenca, Nic",Punho,Feminino,Doença,Rio Grande do Sul,43.0,201801
2,"Veiculo, Nic",414110,S920,4930,Não,Fratura,Pe (Exceto Artelhos),Feminino,Típico,São Paulo,28.0,201801
3,"Asfalto, Alcatrao, P",514205,M255,3811,Não,"Distensao, Torcao","Perna (Do Tornozelo,",Masculino,Típico,São Paulo,30.0,201801
4,Veiculo Rodoviario M,782405,T920,4921,Não,Fratura,Antebraco (Entre O P,Masculino,Típico,Mato Grosso,44.0,201802


In [15]:
df_cat.shape[0]

119216

In [16]:
#apagando os acidentes cujo anoMes não foi preenchido corretamente
condicao = df_cat['anoMes'].str.match('^[0-9]+$') == False
indexItensInvalidos = df_cat[condicao].index

df_cat.drop(indexItensInvalidos, inplace=True)
df_cat.shape[0] #

119216

In [17]:
#apagando as linhas de cujo campo 'cnae' foi preenchido com um código cnae inválido (0)
condicao_cnae_invalido = df_cat['cnae'] == 0
df_cat.drop(df_cat[condicao_cnae_invalido].index, inplace = True)
df_cat.shape[0] #

119213

In [18]:
#separando os acidentes que aconteceram em 2019 dos que aconteceram em 2018:
condicao2018 = df_cat['anoMes'].str.match('^2018[0-9]+') == True
df_cat2018 = df_cat[condicao2018].copy()
df_cat2018.shape[0] #

condicao2017 = df_cat['anoMes'].str.match('^2017[0-9]+') == True
df_cat2017 = df_cat[condicao2017].copy()
df_cat2017.shape[0] #

0

In [19]:
df_cat2018.to_csv(nomeArquivoResultado2018, sep = ';', index=False, encoding='utf-8')

In [20]:
if(df_cat2017.shape[0] > 0):
    df_cat2017.to_csv(nomeArquivoResultado2017, sep = ';', index=False, encoding='UTF-8')